In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
from pathlib import Path

In [ ]:
%ls ../../../data/27_07_results_and_figures/final_results/



In [ ]:
# Getting annotations ... -> not ideal needs to be refactores

# Todo: move file directories to config files, so that can be re-run with different counts


def load_files(samples, results_dir):
    df_list = []
    cntrl_list = []
    for sample in samples:
        df_list.append(pd.read_csv(Path(results_dir)/sample/"merged_counts.csv", index_col=0))
        cntrl_list.append(pd.read_csv(Path(results_dir)/sample/"merged_controls.csv", index_col=0))
    return pd.concat([pd.concat(df_list), pd.concat(cntrl_list)])


counts_dir ="/Users/ansintsova/git_repos/nguyenb_tnseq/data/13_04_results/counts"
outdir = '/Users/ansintsova/git_repos/nguyenb_tnseq/data/07_06_results/'
control_file = Path("/Users/ansintsova/git_repos/nguyenb_tnseq/data/13_04_results")/'controls.txt'

# Load
dnaids = ['dnaid1315', 'dnaid1428', 'dnaid1429', 'dnaid1457', 'dnaid2015', 'dnaid2016', 'dnaid2017', 'dnaid2018', 'dnaid2019',
         'dnaid2023', 'dnaid2024', 'dnaid2025', 'dnaid2026', 'dnaid2027', 'dnaid2028', 'dnaid2029' ]

cnt_df = load_files(dnaids, Path(counts_dir))
# Create unique identifier for each sample
cnt_df['sampleID'] = cnt_df['sampleID'] + "_" + cnt_df['dnaid'] + "_" + cnt_df['experiment']
cnt_df = cnt_df[cnt_df.sampleID.notnull()]
cnt_df['CntrlName'] = cnt_df['phenotype'] + cnt_df['conc'].astype(str)
cnt_df['ShortName'] = cnt_df.ShortName.fillna(cnt_df.CntrlName)

# Dropping Unenriched samples
cnt_df = cnt_df[~cnt_df.sampleID.str.contains('unenriched')]
annotation_df = cnt_df[['barcode', 'ShortName', 'locus_tag', 'phenotype', 'conc']].drop_duplicates()

libraries = [lib for lib in cnt_df.library.unique() if type(lib) == str]
libraries.remove('library_14_1')
print(len(libraries))
days = ['_d1', '_d2', '_d3', '_d4']




In [ ]:
ann = cnt_df[['ShortName', 'locus_tag']].drop_duplicates()
#ann = ann[ann.ShortName]
ann = ann[ann.ShortName.apply(lambda x: len(x) < 10)]
ann.sample(25)

In [ ]:
# Ranking should be done within each day, within each library, then take median rank for each gene
df = pd.read_csv("../../../data/27_07_results_and_figures/final_results/26-07-final-results.csv", index_col=0)
df['dir_change'] = df['z-score'].apply(lambda x: 1 if x > 0 else -1)
df.sample(5)

In [ ]:
# Read in final results table
#df = pd.read_csv("../../results/26-07-final-results.csv", index_col=0)
#df = pd.read_csv("../../../data/27_07_results_and_figures/final_results/26-07-final-results.csv", index_col=0)
# Rank all genes based on direction of change and adjusted p-value

df['dir_change'] = df['z-score'].apply(lambda x: 1 if x > 0 else -1)
df['rank'] = -1*np.log10(df['zscore_padj'])*df['dir_change']
df = df.merge(ann, how='left', left_on = 'gene', right_on ='ShortName').dropna()


In [ ]:
df.sample(10).sort_values('rank', ascending=False)

In [ ]:
# Subset by days
df1 = df[df.day == 'd1']
# Calculate mean, median, min, max rank for each gene
ranked = df1.groupby('locus_tag').agg({'rank':['mean', 'median', 'min', 'max']}).dropna().reset_index()
ranked.columns = ['locus_tag', 'rank_mean', 'rank_median','rank_min', 'rank_max']

# Test these with fgsea

# Mean performed the best
ranked_mean = ranked[['locus_tag', 'rank_mean']]
ranked_mean.to_csv("./day1_ranked_mean.csv")



df2 = df[df.day == 'd2']

ranked = df2.groupby('locus_tag').agg({'rank':['mean', 'median', 'min', 'max']}).dropna().reset_index()
ranked.columns = ['locus_tag', 'rank_mean', 'rank_median','rank_min', 'rank_max']

ranked_mean = ranked[['locus_tag', 'rank_mean']]
ranked_mean.to_csv("./day2_ranked_mean.csv")


df3 = df[df.day == 'd3']

ranked = df3.groupby('locus_tag').agg({'rank':['mean', 'median', 'min', 'max']}).dropna().reset_index()
ranked.columns = ['locus_tag', 'rank_mean', 'rank_median','rank_min', 'rank_max']

ranked_mean = ranked[['locus_tag', 'rank_mean']]
ranked_mean.to_csv("./day3_ranked_mean.csv")



df4 = df[df.day == 'd4']

ranked = df4.groupby('locus_tag').agg({'rank':['mean', 'median', 'min', 'max']}).dropna().reset_index()
ranked.columns = ['locus_tag', 'rank_mean', 'rank_median','rank_min', 'rank_max']

ranked_mean = ranked[['locus_tag', 'rank_mean']]
ranked_mean.to_csv("./day4_ranked_mean.csv")

In [ ]:
# For STRING get all the hits by day
# Day 1
print("\n".join(list(df[(df.zscore_padj < 0.05) & (df.day == 'd1')].gene.unique())))

In [ ]:
l102d1[['gene', 'z-score']].to_csv('../../results/25-08-l102d1-gene-zscore.csv', index=None)
# No enrichment detected
l102d1[['gene', 'log2FC']].to_csv('../../results/25-08-l102d1-gene-log2fc.csv', index=None)
# Nothing
#Try day 2
l102d2[['gene', 'z-score']].to_csv('../../results/25-08-l102d2-gene-zscore.csv', index=None)

In [ ]:
# only hits
l102d1[l102d1.zscore_padj < 0.05][['gene']].to_csv('../../results/25-08-l102d1-gene-hits.csv', index=None, header=None)
# all hits
pd.Series(df1[df1.zscore_padj < 0.05].gene.unique()).to_csv('../../results/25-08-d1-gene-hits.csv', index=None, header=None)

In [ ]:
genes = [g for g in df.gene if len(g) < 10]
# day 1
df1 = df[df.day == 'd1']
#Explore consitency in the resutls
#df1_exp = df1[df1.gene.isin(genes)].groupby(['gene']).agg({'z-score':['mean', 'std', 'median'], 'log2FC':['mean', 'std', 'median']}).dropna()
#df1_exp.columns = ['zscore_mean', 'zscore_std', 'zscore_median', 'fc_mean', 'fc_std', 'fc_median']

In [ ]:
px.scatter(df1_exp, x="zscore_mean", y="normstd", log_y=True, hover_data=[df1_exp.index])

In [ ]:
df1

In [ ]:
ranked = df1.groupby('locus_tag').agg({'rank':['mean', 'median', 'min', 'max', 'std']}).dropna().reset_index()
ranked.columns = ['locus_tag', 'rank_mean', 'rank_median','rank_min', 'rank_max', 'rank_std'
]

In [ ]:
ranked.sort_values('rank_mean').head(40)

In [ ]:
# Try with mean
ranked_mean = ranked[['locus_tag', 'rank_mean']]
ranked_mean.to_csv("./ranked_mean.csv")

ranked_median = ranked[['locus_tag', 'rank_median']]
ranked_median.to_csv("./ranked_median.csv")


ranked_median = ranked[['locus_tag', 'rank_min']]
ranked_median.to_csv("./ranked_min.csv")
# Try with median

# Try with max absolute rank for each gene


# MEAN MAKES MOST SENSE

In [ ]:
df1[df1.gene == 'bapA']

In [ ]:
ranked_mean